In [1]:
from qiskit_ibm_provider import IBMProvider
import pickle

In [3]:
with open('results/rb_experiments/rx_errors.pkl','rb') as f:
    rx_error_rates = pickle.load(f)

with open('results/rb_experiments/rzx_errors.pkl','rb') as f:
    rzx_error_rates = pickle.load(f)

error_rates = {}

In [4]:
def get_system_error_rate(system):
    provider = IBMProvider()
    backend = provider.get_backend(system)
    backend_props = backend.properties()

    system_error_rates = {'sx' : {}, 'rz' : {}, 'x' : {}}

    #Getting the default errors
    for basis_gate in backend.configuration().basis_gates:
        for gate_props in backend_props.gates:
            data = gate_props.to_dict()
            if data['gate'] == basis_gate:
                for param in data['parameters']:
                    if param['name'] == 'gate_error':
                        if basis_gate in ['sx','rz','x']:
                            system_error_rates[basis_gate][data['qubits'][0]] = param['value']

    #Getting the custom errors
    rx_system_error = list(filter(lambda x: x['system'] == system, rx_error_rates))
    rzx_system_error = list(filter(lambda x: x['system'] == system, rzx_error_rates))

    rx_system_error_mapped = dict(map(lambda x: (x['qubit'][0],x['error'].nominal_value + x['error'].std_dev), rx_system_error))
    rzx_system_error_mapped = dict(map(lambda x: ((x['qubit'][0],x['qubit'][1]),x['error'].nominal_value + x['error'].std_dev), rzx_system_error))

    system_error_rates['rx'] = rx_system_error_mapped
    system_error_rates['rzx'] = rzx_system_error_mapped

    return system_error_rates

In [5]:
error_rates = dict(map(lambda x: (x, get_system_error_rate(x)), ['ibm_brisbane','ibm_kyoto','ibm_sherbrooke','ibmq_mumbai','ibm_nazca']))

In [6]:
#Save the results
with open('results/error_rates.pkl','wb') as f:
    pickle.dump(error_rates, f)